In [ ]:
import numpy as np
import matplotlib.pyplot as plot

def getMovieData(filename):
    f_train = open(filename, 'r')
    row = len(f_train.readlines())
    R=np.zeros((row,4))
    counter=0
    for line in open(filename,'r'):
        (R[counter][0],R[counter][1],R[counter][2],R[counter][3])=line.split('\t')
        counter+=1
    return R

train_data = getMovieData('u1.base')
test_data = getMovieData('u1.test')

def getTrueData():
    maxU=-1
    maxM=-1
    for each in train_data:
        if(each[0]>maxU):maxU=each[0]
        if(each[1]>maxM):maxM=each[1]
    R=np.zeros((int(maxU),int(maxM)))
    for each in train_data:
        R[int(each[0]-1)][int(each[1]-1)]=each[2]
    return R
R = getTrueData()

K=8
learning_rate=0.001
penalty_factor=0.5
rows=R.shape[0]     # 用户数量
columns=R.shape[1]  # 电影数量
P=np.ones((rows,K))
Q=np.ones((columns,K))
x = []
y = []

def matrix_factorization(R, P, Q, K, steps=100, alpha=0.002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(R.shape[0]):
            for j in range(R.shape[1]):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P, Q)
        e = 0
        for i in range(R.shape[0]):
            for j in range(R.shape[1]):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in range(K):
                        e = e + (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if e < 0.001:
            break

        loss = test_loss(eR)
        x.append(step)
        y.append(loss)

    return P, Q.T

def test_loss(R):
    loss=0
    for i in test_data:
        userid,movieid,rating=i[0],i[1],i[2]
        predict_rating=R[int(userid)-1][int(movieid)-1]
        loss += pow(predict_rating - rating,2)
    return loss/rows


matrix_factorization(R,P,Q,K)
plot.title('During the training')
plot.xlabel('Rounds')
plot.ylabel('Loss')
plot.plot(np.array(x), np.array(y),label="test")
plot.legend()
plot.show()